# Class-Dependent Perturbation Effects in Evaluating Time Series Attributions

This notebook demonstrates the implementation and analysis of class-dependent effects in perturbation-based evaluation of feature attributions for time series classification, as presented in our paper "Class-Dependent Perturbation Effects in Evaluating Time Series Attributions."

## Overview

Our research examines how perturbation-based evaluation of feature attributions can exhibit varying effectiveness across different predicted classes. These class-dependent effects have important implications for XAI evaluation methodology, particularly when assessing attribution methods using perturbation analysis.

The notebook is organized into two main sections:

### Example Functionality
This section provides practical demonstrations of the key methods employed in our research:
- Loading and preprocessing univariate time series data from the UCR repository
- Training time series classifiers and loading pre-trained models
- Generating feature attributions using various methods (Gradients, Integrated Gradients, SmoothGrad, Gradient SHAP, and Feature Occlusion)
- Using perturbation strategies for attribution evaluation
- Calculating degradation scores (DS)

These examples help build intuition about the building blocks of the methodological approach and the experimental framework used in our investigation of class-dependent perturbation effects.

### Paper Experiment Results
This section demonstrates how to load experimental results from CSV files and reproduce the visualizations presented in the paper.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import os
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from lets_plot import LetsPlot, element_rect, labs, theme

from tsxai.attribution.evaluation.metrics import (
    PerturbationResult,
    calculate_class_adjusted_metric,
    plot_perturbation_analysis_curves,
)
from tsxai.attribution.explainer import TimeSeriesExplainer
from tsxai.data.dataloaders import prepare_dataloaders
from tsxai.data.loading import load_ucr_data, print_dataset_info
from tsxai.modelling.evaluation import evaluate_model
from tsxai.modelling.models import InceptionTime, ResNet
from tsxai.modelling.training import setup_deterministic_training, train_model
from tsxai.modelling.utils import load_trained_model
from tsxai.utils.logging import setup_logger
from tsxai.utils.results import load_and_format_experiment_results
from tsxai.visualization.utils import get_plot_dimensions
from tsxai.visualization.visualize import (
    plot_confusion_matrix,
    plot_metric_distributions,
    plot_parallel_coords,
    plot_perturbation_analysis,
    vis_time_series_classification_samples,
)

2025-02-22 12:13:03.649462: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 12:13:03.670565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740222783.697985    1426 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740222783.704145    1426 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 12:13:03.727061: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

2025-02-22 12:13:11.196 |     INFO |        training | Enabled Tensor Cores with medium precision


In [3]:
while "notebooks" in os.getcwd():
    os.chdir(Path.cwd().parent)
print(f"Working directory: {os.getcwd()}")

Working directory: /home/gbaer/code/xai-ts-rashomon


In [4]:
## Miscellaneous settings & setup
USE_CUDA = True  # Set to False to use CPU for training and inference

logger = setup_logger(__name__, logging.INFO)
LetsPlot.setup_html(no_js=True)
width, height = get_plot_dimensions(scale=2)

device = torch.device("cuda" if USE_CUDA and torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

2025-02-22 12:13:11.383 |     INFO |       815250717 | Using device: cuda


# Example functionality
## Data
First, we import the data from the UCR time series archive. We will use the `"Wafer"` dataset for demonstration here, but you can choose to load any other dataset available. 

`data` will be a custom `UCRDataset` dataclass, containing `X_train`, `X_test`, `y_test`, `y_train`, `n_classes`, `n_features`, `n_timesteps` and more.

We also print some metadata of the dataset, to get an idea of the number of train and test samples, class distributions, etc. And plot 5 samples for each class, to get an idea of the shape of the univariate time series.

In [5]:
dataset_name: str = "Wafer"
n_samples_to_vis: int = 5

data = load_ucr_data(dataset_name, remap_labels_ascending=True, on_disk=False)

# Show dataset metadata and plot some samples
print_dataset_info(data)
ts_samples_plot = vis_time_series_classification_samples(
    data.to_plotting_df(split="train"), n_samples_per_label=n_samples_to_vis
)
ts_samples_plot.show()

2025-02-22 12:13:11.447 |     INFO |         loading | Loading UCR dataset: Wafer
2025-02-22 12:13:11.456 |     INFO |         loading | Dataset loaded: train=1000 test=6164 samples
2025-02-22 12:13:11.461 |     INFO |         loading | Found 2 unique classes
2025-02-22 12:13:11.463 |     INFO |         loading | Successfully processed Wafer data: 1 features, 152 timesteps
╒═══════════════╤═════════╕
│ Metric        │   Value │
╞═══════════════╪═════════╡
│ Train samples │    1000 │
├───────────────┼─────────┤
│ Test samples  │    6164 │
├───────────────┼─────────┤
│ Num features  │       1 │
├───────────────┼─────────┤
│ Num classes   │       2 │
├───────────────┼─────────┤
│ Num timesteps │     152 │
╘═══════════════╧═════════╛

Class Distribution:
╒═════════╤═════════╤════════╕
│ Class   │ Train   │ Test   │
╞═════════╪═════════╪════════╡
│ Class 0 │ 9.7%    │ 10.8%  │
├─────────┼─────────┼────────┤
│ Class 1 │ 90.3%   │ 89.2%  │
╘═════════╧═════════╧════════╛


<path d="M24.3199449585938 110.43583230026731 L24.3199449585938 110.43583230026731 L27.541129721321457 105.52581471721689 L30.762314484049114 107.35753979181263 L33.983499246776766 106.3302267414766 L37.20468400950442 106.9665352744053 L40.42586877223208 106.5695131849321 L43.64705353495973 106.81184359595525 L46.868238297687384 106.59191617437718 L50.089423060415044 106.82548791289162 L53.3106078231427 106.47731308990338 L56.531792585870356 107.03531364511457 L59.75297734859801 106.20539347856442 L62.97416211132566 107.00760589208932 L66.19534687405331 104.0180045029849 L69.41653163678097 108.20347826407551 L72.63771639950863 100.39779704883728 L75.8589011622363 152.81105318370555 L79.08008592496394 192.82075027394757 L82.3012706876916 187.12800472906906 L85.52245545041924 191.04787625874755 L88.7436402131469 188.65022101694532 L91.96482497587456 191.23238046377716 L95.18600973860222 188.21801127424297 L98.40719450132988 192.2605439894738 L101.62837926405751 185.86287062267945 L104.84956402678517 197.237482970054 L108.07074878951283 123.57065942887718 L111.2919335522405 100.01296306669019 L114.51311831496815 108.61771483144129 L117.73430307769581 103.88566686575771 L120.95548784042347 106.5066422527888 L124.1766726031511 105.40776690160627 L127.39785736587876 105.05689074860572 L130.61904212860642 107.01117911230942 L133.84022689133408 103.0877404809386 L137.06141165406174 110.06570725888929 L140.2825964167894 102.03145620159196 L143.50378117951703 119.17710065671534 L146.7249659422447 64.97607036188501 L149.94615070497235 143.11163255561667 L153.1673354677 196.8828408632085 L156.38852023042767 181.18451579747796 L159.6097049931553 194.25189227590553 L162.83088975588296 184.99970237620454 L166.05207451861062 190.04484764514714 L169.27325928133828 188.40772846493616 L172.49444404406594 189.54754900398333 L175.7156288067936 188.95909959530405 L178.93681356952123 188.42218654606648 L182.1579983322489 188.33755812915825 L185.37918309497655 191.99511001757543 L188.6003678577042 190.19516763550257 L191.82155262043187 189.90039226366324 L195.04273738315953 190.5317600854143 L198.2639221458872 190.28330502808925 L201.48510690861482 190.40623236959976 L204.70629167134248 190.37726114058916 L207.92747643407014 190.1720946672662 L211.1486611967978 188.9687240394928 L214.36984595952546 191.03620210775446 L217.59103072225312 189.34610565481097 L220.81221548498075 189.71488767934255 L224.0334002477084 190.53388321455196 L227.25458501043607 190.28178763320125 L230.47576977316373 190.46435226492648 L233.6969545358914 189.73670940864906 L236.91813929861905 189.88824698603156 L240.1393240613467 191.30546015951165 L243.36050882407434 191.5956802874231 L246.581693586802 189.11501192004505 L249.80287834952966 189.67670194750406 L253.02406311225732 190.36028834454365 L256.245247874985 190.41017270148632 L259.46643263771267 190.30387551846596 L262.6876174004403 190.5047284297081 L265.90880216316793 189.59958096417472 L269.1299869258956 190.1940234708088 L272.35117168862325 190.23042259253722 L275.5723564513509 188.45798911133693 L278.7935412140786 188.28742597184075 L282.01472597680623 189.2887597532477 L285.2359107395339 189.2520883570333 L288.45709550226155 190.61844738691855 L291.6782802649892 190.1720273634607 L294.8994650277168 188.81588321569444 L298.1206497904445 187.55216248922062 L301.34183455317213 190.1808319249277 L304.5630193158998 187.95912412706295 L307.78420407862745 188.90038620278327 L311.0053888413551 189.49288913611292 L314.2265736040828 188.8621912931531 L317.44775836681043 188.67464312055586 L320.6689431295381 189.49145740061377 L323.89012789226575 189.05523084442768 L327.1113126549934 188.1678577020782 L330.3324974177211 188.3016882600823 L333.5536821804487 189.06627478706002 L336.77486694317633 188.6910377157267 L339.996051705904 190.71524249630565 L343.21723646863165 187.9532656367192 L346.4384212313593 191.0514617160232 L349.659605994087 184.88661362735485 L352.88079075681463 195.71124680943012 L356.1019755195423 169.48222

## Classifiers
### Load trained model (or train from scratch)
Next, we need trained classifiers to generate predictions for our time series data. This example demonstrates loading a pre-trained ResNet model from our experimental results.

Alternative options include:
- Switching to the InceptionTime architecture by setting `model_name = "InceptionTime"`
- Training a new model from scratch by setting `load_model = False`

The code maintains consistent data preparation settings with those used in our training script (`scripts/train_models.py`), ensuring compatibility with the evaluation framework described in our paper. This consistency is important for achieving the same results.

In [6]:
# set to True to load a pre-trained model, otherwise train a new model from scratch
load_model: bool = True
dataset_name: str = "Wafer"
model_name: str = "ResNet"  # either "ResNet" or "InceptionTime"
model_dir: str = "results/models"  # Path to previously trained models


data = load_ucr_data(dataset_name, remap_labels_ascending=True, on_disk=False)

# use same settings as in scripts/train_models.py for dataloaders
train_loader, val_loader, test_loader = prepare_dataloaders(
    data, batch_size=256, val_split=0.2, seed=42
)

if load_model:
    model = load_trained_model(data, dataset_name, model_name, model_dir)
else:
    config = {
        "dataset_name": dataset_name,
        "model_dir": "models",
        "save_model": False,
        "save_hyperparameters": False,
        "show_progress": True,
        "patience": 25,
        "epochs": 500,
    }

    # initialise models
    setup_deterministic_training()
    c_in, c_out = (data.n_features, data.n_classes)
    resnet = ResNet(c_in, c_out)
    inceptiontime = InceptionTime(c_in, c_out)
    models_dict = {"ResNet": resnet, "InceptionTime": inceptiontime}

    # train selected model
    assert model_name in models_dict.keys(), (
        f"Model {model_name} not found in models_dict"
    )
    model_to_train = models_dict[model_name]

    model = train_model(
        model=model_to_train,
        train_loader=train_loader,
        val_loader=val_loader,
        config=config,
    )

2025-02-22 12:13:11.631 |     INFO |         loading | Loading UCR dataset: Wafer
2025-02-22 12:13:11.637 |     INFO |         loading | Dataset loaded: train=1000 test=6164 samples
2025-02-22 12:13:11.642 |     INFO |         loading | Found 2 unique classes
2025-02-22 12:13:11.643 |     INFO |         loading | Successfully processed Wafer data: 1 features, 152 timesteps
2025-02-22 12:13:11.645 |     INFO |     dataloaders | Preparing dataloaders (batch_size=256, val_split=0.2)
2025-02-22 12:13:11.648 |     INFO |     dataloaders | Created dataloaders - Train: 800, Val: 200, Test: 6164
2025-02-22 12:13:11.650 |     INFO |           utils | Loading trained ResNet model for dataset Wafer
2025-02-22 12:13:11.651 |     INFO |           utils | Using device cuda for loading weights
2025-02-22 12:13:12.564 |     INFO |           utils | Successfully loaded trained ResNet model for Wafer on cuda


## Classifiers
### Evaluate Model
We evaluate the loaded model's performance using standard classification metrics. The code calculates accuracy and F1 scores, while also generating a confusion matrix to visualize prediction patterns across classes. This validation step ensures that our model achieves sufficient classification capability before proceeding to attribution analysis.

For the Wafer dataset with ResNet architecture, we observe high predictive performance (Accuracy: 0.993, F1: 0.993), which match the results reported in our paper. 

In [7]:
# compute accuracy and f1 score and display confusion matrix
perf_metrics = evaluate_model(model, test_loader, device, data.n_classes, True)
print(
    f"Accuracy: {perf_metrics[0]['accuracy']:.3f} | F1: {perf_metrics[0]['f1']:.3f}",
    f"[Data: {dataset_name}, Model: {model_name}]",
)
conf_matrix_plot = plot_confusion_matrix(perf_metrics[1])
conf_matrix_plot.show()

Accuracy: 0.993 | F1: 0.993 [Data: Wafer, Model: ResNet]


625 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 5,496 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Predicted 0 
 
 
 
 
 
 
 Predicted 1 
 
 
 
 
 
 
 
 
 Actual 0 
 
 
 
 
 
 
 Actual 1 
 
 
 
 
 
 
 
 
 
 
 
 
 Count 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 
 
 5,000

## Feature Attributions

Now we move to generating feature attributions to explain the model's classifications. This section demonstrates using our custom `TimeSeriesExplainer` class to:
1. Generate attributions for specific samples
2. Visualize attribution results 
3. Perform perturbation analysis

Our implementation builds on the [TSInterpret](https://github.com/fzi-forschungszentrum-informatik/TSInterpret) library with additional functionality for working with `UCRDataset` objects and other visualization capabilities. 

We initialize the explainer with `split="test"` to ensure we're generating explanations for previously unseen data.

In [8]:
# initialise explainer to explain test set observations on trained model
explainer = TimeSeriesExplainer(dataset=data, model=model, split="test")

### Generating Feature Attributions

To produce feature attributions, we call the explainer's `.explain()` method with three key parameters:
- `sample_index`: Specifies which instance to explain
- `method`: Indicates which attribution algorithm to apply
- `label`: Determines which class prediction to explain (can be "predicted", "true", or a specific class index)

The underlying TSInterpret implementation supports multiple attribution methods:

```python
available_attribution_methods = ["GRAD", "IG", "GS", "DL", "DLS", "SG", "SVS", "FA", "FO"]
```

These correspond to:
* Gradients (`GRAD`)
* Integrated Gradients (`IG`)
* Gradient Shap (`GS`)
* DeepLift (`DL`)
* DeepLiftShap (`DLS`)
* SmoothGrad (`SG`)
* Shapley Value Sampling(`SVS`)
* Feature Ablation (`FA`)
* Feature Occlusion (`FO`)

Note: For the analysis in our paper, we specifically focus on five methods: Gradients, Integrated Gradients, SmoothGrad, Gradient SHAP, and Feature Occlusion.

The `.explain()` method returns an `ExplanationResult` dataclass containing:
```
ExplanationResult(
    metadata=Metadata(true_label, observation_index), 
    prediction=Prediction(label, probability, all_probabilities, explained_label), 
    data=Data(ds, y, attributions, method)
)
```

This structured output provides all information needed for subsequent perturbation analysis and evaluation.

In [9]:
# Configuration for explanation
sample_idx: int = 0  # which sample to explain
method: str = "GS"  # attribution method to use
label: str | int = "predicted"  # label to explain

# explain sample
explanation = explainer.explain(sample_index=sample_idx, method=method, label=label)
print(explanation)

ExplanationResult(metadata=Metadata(true_label=1, observation_index=0), prediction=Prediction(label=1, probability=0.9961244463920593, all_probabilities=array([0.00387554, 0.99612445], dtype=float32), explained_label=1), data=Data(ds=array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137,

### Visualizing Feature Attributions

Our implementation includes built-in visualization capabilities through the `.visualize_attributions()` method, which provides an intuitive representation of feature importance within the time series.

The visualization consists of:
- A white line showing the original time series signal
- A heatmap overlay indicating attribution values at each time point
- A comprehensive title displaying the explained label, prediction confidence, and correctness

In this example, we visualize sample 1 using the Feature Occlusion (`FO`) method to explain the predicted label. The classifier assigns this prediction with 99.5% confidence, and the prediction matches the true label.

The resulting visualization reveals that certain constant periods at approximately time steps 0-20, 34-49, and 122-150 (where the signal remains around -1) contribute most significantly to the model's prediction. These highlighted regions align with our understanding of discriminative patterns in the Wafer dataset, where the sustained periods at specific values often characterize class membership.

In [10]:
sample_idx: int = 1
method: str = "FO"
label: str | int = "predicted"

explanation = explainer.explain(sample_index=sample_idx, method=method, label=label)
plot = explanation.visualize_attributions()
plot.show()

<path d="M2.3979759648824026 183.79974215085426 L2.3979759648824026 183.79974215085426 L7.193927894647208 183.62897710550402 L11.989879824412013 183.62897710550402 L16.78583175417682 183.62897710550402 L21.581783683941623 183.62897710550402 L26.377735613706427 183.62897710550402 L31.173687543471235 183.79974215085426 L35.96963947323604 183.62897710550402 L40.76559140300085 183.62897710550402 L45.56154333276565 183.79974215085426 L50.357495262530456 183.79974215085426 L55.15344719229526 183.79974215085426 L59.94939912206007 183.79974215085426 L64.74535105182487 183.79974215085426 L69.54130298158968 183.79974215085426 L74.33725491135448 183.79974215085426 L79.13320684111929 183.79974215085426 L83.9291587708841 183.79974215085426 L88.7251107006489 183.79974215085426 L93.52106263041371 183.79974215085426 L98.3170145601785 41.04266132263621 L103.11296648994332 8.939393939393923 L107.90891841970812 18.843588874550306 L112.70487034947293 41.04266132263621 L117.50082227923774 55.215912248723214 L122.29677420900254 68.36458926936803 L127.09272613876735 76.21964340792255 L131.88867806853213 83.90393600826809 L136.68462999829694 88.51451156847541 L141.48058192806175 92.10051497681117 L146.27653385782654 95.51575567789088 L151.07248578759135 97.56490042002665 L155.86843771735616 99.61404501603154 L160.66438964712097 172.3586805122992 L165.46034157688578 183.62897710550402 L170.25629350665056 183.62897710550402 L175.05224543641538 183.62897710550402 L179.8481973661802 183.62897710550402 L184.644149295945 183.79974215085426 L189.44010122570978 183.79974215085426 L194.2360531554746 183.62897710550402 L199.0320050852394 183.79974215085426 L203.82795701500422 183.62897710550402 L208.62390894476903 183.79974215085426 L213.4198608745338 183.79974215085426 L218.21581280429862 183.79974215085426 L223.01176473406343 183.62897710550402 L227.80771666382824 183.62897710550402 L232.60366859359306 183.79974215085426 L237.39962052335784 183.79974215085426 L242.19557245312265 68.87687505304204 L246.99152438288746 38.82275314258995 L251.78747631265227 31.821507864127398 L256.5834282424171 27.040170765711096 L261.37938017218187 25.332553045527177 L266.17533210194665 24.307976801990833 L270.9712840317115 23.79569101831683 L275.7672359614763 23.283405234642814 L280.5631878912411 22.600359081806914 L285.3591398210059 21.575787514458895 L290.1550917507707 21.405022469108673 L294.9510436805355 20.380455577948993 L299.7469956103003 20.89273668543467 L304.54294754006514 21.063501730784893 L309.3388994698299 21.063501730784893 L314.1348513995947 21.23426209994679 L318.93080332935955 21.91730825278269 L323.72675525912433 22.600359081806914 L328.52270718888917 22.600359081806914 L333.31865911865395 23.283405234642814 L338.11461104841874 23.79569101831683 L342.9105629781836 24.307976801990833 L347.70651490794836 24.649502216502952 L352.5024668377132 24.820267261853175 L357.298418767478 25.161788000176955 L362.09437069724277 25.332553045527177 L366.8903226270076 25.503313414689075 L371.6862745567724 25.844838829201194 L376.4822264865372 26.357124612875197 L381.278178416302 26.698650027387316 L386.0741303460668 26.698650027387316 L390.87008227583163 26.357124612875197 L395.6660342055964 26.527884982037094 L400.4619861353612 26.357124612875197 L405.25793806512604 26.698650027387316 L410.0538899948908 26.357124612875197 L414.84984192465566 26.698650027387316 L419.64579385442045 26.698650027387316 L424.44174578418523 26.357124612875197 L429.23769771395007 26.015599198363077 L434.03364964371485 26.527884982037094 L438.8296015734797 26.015599198363077 L443.6255535032445 26.357124612875197 L448.42150543300926 25.67407378385097 L453.2174573627741 25.844838829201194 L458.0134092925389 25.503313414689075 L462.8093612223037 25.503313414689075 L467.6053131520685 25.332553045527177 L472.4012650818333 24.478741847341055 L477.1972170115981 21.746547883620792 L481.9931689413629 17.64826161422873 L486.7891208711277 12.525403777488634 L491.58507280089253 12.696159470462206 L496.38102473

### Evaluating attributions with perturbations

To assess attribution quality, we can measure how perturbing input features affects model predictions. The `plot_perturbation_analysis()` function helps to:
1. Generate feature attributions for a single observation
2. Evaluate them using perturbation
3. Visualize the results

In this example, we configure the perturbation analysis with several parameters:
- `perturbation_ratio = 0.5`: We perturb the first 50% of time points
- `auc_types=["MoRF", "LeRF"]`: We use two perturbation orders:
  - Most Relevant Features first (MoRF): Features perturbed in descending importance order
  - Least Relevant Features first (LeRF): Features perturbed in ascending importance order
- `perturbation_strategy = "zero"`: We replace feature values with zeros
- `features_per_step`: Controls how many features are perturbed in each step

For computational efficiency, we could increase `features_per_step`, though this reduces granularity of the results. With Wafer's time series length of 152, a perturbation ratio of 0.5 and `features_per_step = 1` means we'll perform 76 perturbations operations.

The example results below show effective feature attributions:
- The MoRF curve decreases relatively quickly (desirable), yielding a low area under the curve ($AUC_{MoRF} = 0.043$)
- The LeRF curve remains relatively unchanged until ~10% of features are perturbed (desirable), producing a higher AUC ($AUC_{LeRF} = 0.338$)

The degradation score (DS = 0.305) represents the difference between these curves, visualized as the grey area between them. Higher DS values generally indicate better attribution quality.

In [11]:
# Configuration for perturbation analysis
sample_idx: int = 1
method: str = "FO"
label: str | int = "predicted"
perturbation_strategy: str | int = "zero"
perturbation_ratio: float = 0.5
features_per_step: int = 1


# Run perturbation analysis for sample and show results
plot_perturbation_analysis(
    explainer=explainer,
    sample_idx=sample_idx,
    method=method,
    label=label,
    perturbation_strategy=perturbation_strategy,
    show_plot=True,
    auc_types=["MoRF", "LeRF"],
    plot_between=True,
    perturbation_ratio=perturbation_ratio,
    features_per_step=features_per_step,
)

╒════════════╤════════════╤════════════╕
│   AUC MoRF │   AUC LeRF │   DS Score │
╞════════════╪════════════╪════════════╡
│      0.043 │      0.338 │      0.305 │
╘════════════╧════════════╧════════════╛


<path d="M2.3979759648824026 181.93059223067607 L2.3979759648824026 181.93059223067607 L7.193927894647208 181.76156377900736 L11.989879824412013 181.76156377900736 L16.78583175417682 181.76156377900736 L21.581783683941623 181.76156377900736 L26.377735613706427 181.76156377900736 L31.173687543471235 181.93059223067607 L35.96963947323604 181.76156377900736 L40.76559140300085 181.76156377900736 L45.56154333276565 181.93059223067607 L50.357495262530456 181.93059223067607 L55.15344719229526 181.93059223067607 L59.94939912206007 181.93059223067607 L64.74535105182487 181.93059223067607 L69.54130298158968 181.93059223067607 L74.33725491135448 181.93059223067607 L79.13320684111929 181.93059223067607 L83.9291587708841 181.93059223067607 L88.7251107006489 181.93059223067607 L93.52106263041371 181.93059223067607 L98.3170145601785 40.62527832613483 L103.11296648994332 8.848484848484844 L107.90891841970812 18.651959157182006 L112.70487034947293 40.62527832613483 L117.50082227923774 54.65439449704129 L122.29677420900254 67.66935615815413 L127.09272613876735 75.44452839021487 L131.88867806853213 83.05067564208231 L136.68462999829694 87.61436399320279 L141.48058192806175 91.16389957026733 L146.27653385782654 94.54440900998013 L151.07248578759135 96.57271499202639 L155.86843771735616 98.60102082942784 L160.66438964712097 170.60588037149614 L165.46034157688578 181.76156377900736 L170.25629350665056 181.76156377900736 L175.05224543641538 181.76156377900736 L179.8481973661802 181.76156377900736 L184.644149295945 181.93059223067607 L189.44010122570978 181.93059223067607 L194.2360531554746 181.76156377900736 L199.0320050852394 181.93059223067607 L203.82795701500422 181.76156377900736 L208.62390894476903 181.93059223067607 L213.4198608745338 181.93059223067607 L218.21581280429862 181.93059223067607 L223.01176473406343 181.76156377900736 L227.80771666382824 181.76156377900736 L232.60366859359306 181.93059223067607 L237.39962052335784 181.93059223067607 L242.19557245312265 68.17643225589246 L246.99152438288746 38.42794548351278 L251.78747631265227 31.497899309576965 L256.5834282424171 26.765185978263204 L261.37938017218187 25.074933862013353 L266.17533210194665 24.06077703790281 L270.9712840317115 23.55370094016446 L275.7672359614763 23.046624842426127 L280.5631878912411 22.370524921652958 L285.3591398210059 21.356372726176275 L290.1550917507707 21.18734427450758 L294.9510436805355 20.173196707664772 L299.7469956103003 20.680268176769246 L304.54294754006514 20.84929662843794 L309.3388994698299 20.84929662843794 L314.1348513995947 21.01832045147276 L318.93080332935955 21.69442037224593 L323.72675525912433 22.370524921652958 L328.52270718888917 22.370524921652958 L333.31865911865395 23.046624842426127 L338.11461104841874 23.55370094016446 L342.9105629781836 24.06077703790281 L347.70651490794836 24.398829312606324 L352.5024668377132 24.56785776427502 L357.298418767478 24.90590541034466 L362.09437069724277 25.074933862013353 L366.8903226270076 25.243957685048187 L371.6862745567724 25.5820099597517 L376.4822264865372 26.089086057490036 L381.278178416302 26.42713833219355 L386.0741303460668 26.42713833219355 L390.87008227583163 26.089086057490036 L395.6660342055964 26.258109880524856 L400.4619861353612 26.089086057490036 L405.25793806512604 26.42713833219355 L410.0538899948908 26.089086057490036 L414.84984192465566 26.42713833219355 L419.64579385442045 26.42713833219355 L424.44174578418523 26.089086057490036 L429.23769771395007 25.75103378278652 L434.03364964371485 26.258109880524856 L438.8296015734797 25.75103378278652 L443.6255535032445 26.089086057490036 L448.42150543300926 25.412981508083007 L453.2174573627741 25.5820099597517 L458.0134092925389 25.243957685048187 L462.8093612223037 25.243957685048187 L467.6053131520685 25.074933862013353 L472.4012650818333 24.229805489571504 L477.1972170115981 21.52539654921111 L481.9931689413629 17.468787767304377 L486.7891208711277 12.398026789920976 L491.58507280089253 12.567045984321922 L496.3810247306573 14.7643788

### Alternative Perturbation Strategies

We can evaluate the same attribution using different perturbation strategies to assess robustness. For example, instead of zero replacement, we can use a constant value of 1. This choice is informative since we observed that relevant features in our example cluster around -1.

This alternative strategy yields a higher degradation score, suggesting improved discrimination between relevant and irrelevant features for this particular instance. However, we also notice that the MoRF perturbation curve increases after 40% perturbation, which may indicate some limitations in either the attribution method or perturbation strategy.

The implementation supports various perturbation approaches:
```python
available_perturbation_strategies = [
    "zero",             # Replace with 0
    "mean",             # Replace with instance mean
    "gaussian_noise",   # Replace with random gaussian noise
    "opposite",         # Replace with negated values
    "uniform_noise",    # Replace with uniform noise
    "subsequence_mean", # Replace with local mean
    "inverse",          # Replace with inverted values
    "ood_high",         # Replace with high out-of-distribution value
    "ood_low",          # Replace with low out-of-distribution value
]
```

Beyond these predefined strategies, you can also specify custom constant values as demonstrated below. Different perturbation strategies may perform better for different instances, highlighting the importance of considering multiple approaches when evaluating attributions.

In [12]:
perturbation_strategy: str | int = 1

plot_perturbation_analysis(
    explainer=explainer,
    sample_idx=sample_idx,
    method=method,
    label=label,
    perturbation_strategy=perturbation_strategy,
    show_plot=True,
    auc_types=["MoRF", "LeRF"],
    plot_between=True,
    perturbation_ratio=perturbation_ratio,
    features_per_step=features_per_step,
)

╒════════════╤════════════╤════════════╕
│   AUC MoRF │   AUC LeRF │   DS Score │
╞════════════╪════════════╪════════════╡
│      0.123 │      0.751 │      0.819 │
╘════════════╧════════════╧════════════╛


<path d="M2.3979759648824026 181.93059223067607 L2.3979759648824026 181.93059223067607 L7.193927894647208 181.76156377900736 L11.989879824412013 181.76156377900736 L16.78583175417682 181.76156377900736 L21.581783683941623 181.76156377900736 L26.377735613706427 181.76156377900736 L31.173687543471235 181.93059223067607 L35.96963947323604 181.76156377900736 L40.76559140300085 181.76156377900736 L45.56154333276565 181.93059223067607 L50.357495262530456 181.93059223067607 L55.15344719229526 181.93059223067607 L59.94939912206007 181.93059223067607 L64.74535105182487 181.93059223067607 L69.54130298158968 181.93059223067607 L74.33725491135448 181.93059223067607 L79.13320684111929 181.93059223067607 L83.9291587708841 181.93059223067607 L88.7251107006489 181.93059223067607 L93.52106263041371 181.93059223067607 L98.3170145601785 40.62527832613483 L103.11296648994332 8.848484848484844 L107.90891841970812 18.651959157182006 L112.70487034947293 40.62527832613483 L117.50082227923774 54.65439449704129 L122.29677420900254 67.66935615815413 L127.09272613876735 75.44452839021487 L131.88867806853213 83.05067564208231 L136.68462999829694 87.61436399320279 L141.48058192806175 91.16389957026733 L146.27653385782654 94.54440900998013 L151.07248578759135 96.57271499202639 L155.86843771735616 98.60102082942784 L160.66438964712097 170.60588037149614 L165.46034157688578 181.76156377900736 L170.25629350665056 181.76156377900736 L175.05224543641538 181.76156377900736 L179.8481973661802 181.76156377900736 L184.644149295945 181.93059223067607 L189.44010122570978 181.93059223067607 L194.2360531554746 181.76156377900736 L199.0320050852394 181.93059223067607 L203.82795701500422 181.76156377900736 L208.62390894476903 181.93059223067607 L213.4198608745338 181.93059223067607 L218.21581280429862 181.93059223067607 L223.01176473406343 181.76156377900736 L227.80771666382824 181.76156377900736 L232.60366859359306 181.93059223067607 L237.39962052335784 181.93059223067607 L242.19557245312265 68.17643225589246 L246.99152438288746 38.42794548351278 L251.78747631265227 31.497899309576965 L256.5834282424171 26.765185978263204 L261.37938017218187 25.074933862013353 L266.17533210194665 24.06077703790281 L270.9712840317115 23.55370094016446 L275.7672359614763 23.046624842426127 L280.5631878912411 22.370524921652958 L285.3591398210059 21.356372726176275 L290.1550917507707 21.18734427450758 L294.9510436805355 20.173196707664772 L299.7469956103003 20.680268176769246 L304.54294754006514 20.84929662843794 L309.3388994698299 20.84929662843794 L314.1348513995947 21.01832045147276 L318.93080332935955 21.69442037224593 L323.72675525912433 22.370524921652958 L328.52270718888917 22.370524921652958 L333.31865911865395 23.046624842426127 L338.11461104841874 23.55370094016446 L342.9105629781836 24.06077703790281 L347.70651490794836 24.398829312606324 L352.5024668377132 24.56785776427502 L357.298418767478 24.90590541034466 L362.09437069724277 25.074933862013353 L366.8903226270076 25.243957685048187 L371.6862745567724 25.5820099597517 L376.4822264865372 26.089086057490036 L381.278178416302 26.42713833219355 L386.0741303460668 26.42713833219355 L390.87008227583163 26.089086057490036 L395.6660342055964 26.258109880524856 L400.4619861353612 26.089086057490036 L405.25793806512604 26.42713833219355 L410.0538899948908 26.089086057490036 L414.84984192465566 26.42713833219355 L419.64579385442045 26.42713833219355 L424.44174578418523 26.089086057490036 L429.23769771395007 25.75103378278652 L434.03364964371485 26.258109880524856 L438.8296015734797 25.75103378278652 L443.6255535032445 26.089086057490036 L448.42150543300926 25.412981508083007 L453.2174573627741 25.5820099597517 L458.0134092925389 25.243957685048187 L462.8093612223037 25.243957685048187 L467.6053131520685 25.074933862013353 L472.4012650818333 24.229805489571504 L477.1972170115981 21.52539654921111 L481.9931689413629 17.468787767304377 L486.7891208711277 12.398026789920976 L491.58507280089253 12.567045984321922 L496.3810247306573 14.7643788

# Paper Experiment Results

## Classifier Performance

This section provides access to the experimental results presented in our paper. We begin by examining the classifier performance metrics across all datasets and architectures.

The table displays training, validation, and test accuracy and F1 scores for each model configuration. These results correspond to Table 1 in our paper, showing the classification performance that forms the foundation for our attribution analysis.

The trained model weights are stored in the `results/models` directory and can be loaded using the procedure demonstrated in the "Load trained model" section above.

In [13]:
# check performance metrics of classifiers for all datasets and models
performance_fname = "results/models/performance.csv"
performance_df = pd.read_csv(performance_fname)
performance_df.round(4)

,dataset,model,train_accuracy,val_accuracy,test_accuracy,train_f1,val_f1,test_f1
0,Wafer,ResNet,1.0000,1.0000,0.9925,1.0000,1.0000,0.9924
1,Wafer,InceptionTime,1.0000,1.0000,0.9982,1.0000,1.0000,0.9982
2,FordA,ResNet,0.9990,0.9307,0.9371,0.9990,0.9306,0.9371
3,FordA,InceptionTime,1.0000,0.9445,0.9523,1.0000,0.9445,0.9523
4,FordB,ResNet,1.0000,0.9299,0.8037,1.0000,0.9299,0.8037
5,FordB,InceptionTime,0.9966,0.9382,0.8494,0.9966,0.9381,0.8493
6,ElectricDevices,ResNet,0.9807,0.9127,0.7157,0.9808,0.9127,0.7005
7,ElectricDevices,InceptionTime,0.9874,0.8953,0.7024,0.9875,0.8956,0.6894


## Perturbation Results

This section shows how to access the perturbation analysis results from our experiments. The data is stored in CSV format at `results/perturbation_results/paper_results/compiled_results.csv` for each experimental configuration.

The `load_and_format_experiment_results()` utility function simplifies data loading and preprocessing. This function offers filtering options through the `perturbation_mode` parameter:
- `perturbation_mode = "named"`: Loads results for named strategies (Gauss, Inv, Opp, SubMean, Unif, Zero)
- `perturbation_mode = "constant"`: Loads results for constant-value perturbations

Note that "zero" in named strategies and 0 in constant strategies represent identical perturbation approaches. We include zero in the named strategies for consistency with prior literature.

The results contain detailed metrics for each experimental condition:
- Instance identifiers and class labels (true and predicted)
- Evaluation metrics (AUC for MoRF and LeRF perturbation orders)
- Calculated degradation scores (DS)
- Experimental metadata (dataset, classifier, attribution method)

This dataset enables reproduction of all analyses presented in our paper and supports additional investigations into class-dependent perturbation effects.

In [14]:
# where the results from perturbation experiments are saved
results_fname = "compiled_results.csv"

RESULTS_DIR = Path.cwd() / "results/perturbation_results/paper_results"
RESULTS_PATH = RESULTS_DIR / results_fname
assert RESULTS_PATH.exists(), f"Results file not found at: {RESULTS_PATH}"

In [15]:
# load results for "named" perturbation strategies separately
results_df_named = load_and_format_experiment_results(
    RESULTS_PATH, perturbation_mode="named"
)

print(f"Classifiers: {results_df_named['model_name'].unique()}")
print(f"Attribution Methods: {results_df_named['attribution_method'].unique()}")
print(f"Datasets: {results_df_named['dataset_name'].unique()}")
print(f"Perturbation strategies: {results_df_named['perturbation_strategy'].unique()}")
print(f"Results dimensions: {results_df_named.shape}")
results_df_named.drop(columns="probabilities")

Classifiers: ['ResNet' 'InceptionTime']
Attribution Methods: ['GS' 'FO' 'IG' 'SG' 'GR']
Datasets: ['Wafer' 'FordB' 'FordA' 'ElectricDevices']
Perturbation strategies: ['Inv' 'SubMean' 'Unif' 'Gauss' 'Opp' 'Zero']
Results dimensions: (234000, 12)


,idx,true_label,predicted_label,auc_morf,auc_lerf,ds,dataset_name,model_name,attribution_method,perturbation_strategy,n_attributions
2100,2625,0,0,0.979224,0.936108,-0.000774,Wafer,ResNet,GS,Inv,300
2101,2674,0,0,0.915738,0.858674,0.000150,Wafer,ResNet,GS,Inv,300
2102,4246,0,0,0.941799,0.882541,-0.000009,Wafer,ResNet,GS,Inv,300
2103,2167,0,0,0.939960,0.903231,-0.003626,Wafer,ResNet,GS,Inv,300
2104,3912,0,0,0.931968,0.932883,0.010144,Wafer,ResNet,GS,Inv,300
...,...,...,...,...,...,...,...,...,...,...,...
545995,1675,6,4,0.169130,0.267614,0.082983,ElectricDevices,ResNet,IG,Opp,300
545996,1700,6,3,0.301534,0.109045,-0.137937,ElectricDevices,ResNet,IG,Opp,300
545997,1434,6,6,0.307993,0.434978,0.140919,ElectricDevices,ResNet,IG,Opp,300
545998,1693,6,4,0.400427,0.374040,0.052629,ElectricDevices,ResNet,IG,Opp,300


In [16]:
# load results for constant perturbation strategies separately
results_df_constant = load_and_format_experiment_results(
    RESULTS_PATH, perturbation_mode="constant"
)

print(f"Classifiers: {results_df_constant['model_name'].unique()}")
print(f"Attribution Methods: {results_df_constant['attribution_method'].unique()}")
print(f"Datasets: {results_df_constant['dataset_name'].unique()}")
print(
    f"Perturbation strategies: {results_df_constant['perturbation_strategy'].unique()}"
)
print(f"Results dimensions: {results_df_constant.shape}")
results_df_constant.drop(columns="probabilities")

Classifiers: ['InceptionTime' 'ResNet']
Attribution Methods: ['FO' 'SG' 'IG' 'GS' 'GR']
Datasets: ['ElectricDevices' 'FordB' 'FordA' 'Wafer']
Perturbation strategies: ['1.5' '-1' '1' '2' '-0.5' '0' '0.5' '-2' '-1.5']
Results dimensions: (351000, 12)


,idx,true_label,predicted_label,auc_morf,auc_lerf,ds,dataset_name,model_name,attribution_method,perturbation_strategy,n_attributions
0,7381,0,5,0.179608,0.066264,-0.131603,ElectricDevices,InceptionTime,FO,1.5,300
1,496,0,4,0.527483,0.801977,0.284112,ElectricDevices,InceptionTime,FO,1.5,300
2,7355,0,0,0.021424,0.155690,0.084731,ElectricDevices,InceptionTime,FO,1.5,300
3,3240,0,0,0.022714,0.150046,0.117669,ElectricDevices,InceptionTime,FO,1.5,300
4,7356,0,3,0.376539,0.668450,0.308652,ElectricDevices,InceptionTime,FO,1.5,300
...,...,...,...,...,...,...,...,...,...,...,...
543895,573,1,1,0.076219,0.156905,0.077213,FordB,ResNet,SG,-0.5,300
543896,178,1,1,0.053277,0.080145,0.023871,FordB,ResNet,SG,-0.5,300
543897,809,1,1,0.051478,0.090335,0.035659,FordB,ResNet,SG,-0.5,300
543898,172,1,1,0.088861,0.085707,-0.002937,FordB,ResNet,SG,-0.5,300


### Class-adjusted Correctness Metrics

The `calculate_class_adjusted_metric()` function aggregates the degradation score (DS) metric over all experimental conditions. The parameter α controls the strength of class-specific penalization:
- `α = 0`: Equivalent to taking the mean across experimental conditions
- `α > 0`: Penalizes the average metric with the difference of average metrics between class pairs

The example below calculates results with `α = 1` for the named perturbation strategies.

In [17]:
alpha: float = 1  # weight for penalty term in class-adjusted metric

experimental_conditions = [
    "dataset_name",
    "model_name",
    "attribution_method",
    "perturbation_strategy",
]

aggregate_ds_results = calculate_class_adjusted_metric(
    df=results_df_named,
    metric_col="ds",
    group_cols=experimental_conditions,
    class_col="true_label",
    alpha=alpha,
)
aggregate_ds_results


,dataset_name,model_name,attribution_method,perturbation_strategy,adjusted_metric
0,ElectricDevices,InceptionTime,FO,Gauss,-0.091163
1,ElectricDevices,InceptionTime,FO,Inv,-0.035715
2,ElectricDevices,InceptionTime,FO,Opp,0.122114
3,ElectricDevices,InceptionTime,FO,SubMean,0.229013
4,ElectricDevices,InceptionTime,FO,Unif,0.022093
...,...,...,...,...,...
235,Wafer,ResNet,SG,Inv,-0.000187
236,Wafer,ResNet,SG,Opp,-0.000804
237,Wafer,ResNet,SG,SubMean,-0.005194
238,Wafer,ResNet,SG,Unif,0.001403


### Aggregate Experimental Results

Pre-computed aggregated results from `scripts/aggregate_experiment_results.py` are available in four tables:
1. `average_ds_table.csv`: Mean DS for named strategies
2. `adjusted_ds_table.csv`: Class-adjusted DS for named strategies (α=1)
3. `constant_average_ds_table.csv`: Mean DS for constant-value strategies
4. `constant_adjusted_ds_table.csv`: Class-adjusted DS for constant-value strategies (α=1)

The example below loads `adjusted_ds_table.csv`, reproducing the results presented in our paper.

In [18]:
agg_results_paths = sorted([f for f in (RESULTS_DIR / "tables").iterdir()])
print("Aggregated results tables:\n", [f.name for f in agg_results_paths])


Aggregated results tables:
 ['adjusted_ds_table.csv', 'average_ds_table.csv', 'constant_adjusted_ds_table.csv', 'constant_average_ds_table.csv']


In [19]:
fname: str = "adjusted_ds_table.csv"
file_path = [f for f in agg_results_paths if fname in f.name][0]


agg_results_df = pd.read_csv(file_path, index_col=[0, 1], header=[0, 1]).round(2)
agg_results_df

Model                        ResNet                         InceptionTime  \
Attribution                      GR    IG    SG    GS    FO            GR   
Dataset         Perturbation                                                
FordA           Gauss         -0.01  0.00  0.00 -0.08  0.00         -0.01   
                Inv           -0.00  0.00  0.00  0.00  0.00         -0.01   
                Opp           -0.05  0.00  0.00  0.00  0.00         -0.10   
                SubMean       -0.08  0.00  0.00  0.00  0.01         -0.19   
                Unif          -0.02  0.00  0.00  0.00  0.00         -0.03   
                Zero          -0.08  0.00  0.00  0.00  0.01         -0.20   
FordB           Gauss         -0.00  0.00 -0.00 -0.05  0.01         -0.07   
                Inv           -0.00  0.00 -0.00  0.00  0.00         -0.01   
                Opp           -0.02  0.00  0.00  0.00  0.00         -0.09   
                SubMean       -0.04  0.01  0.00  0.00  0.01         -0.16   
                Unif          -0.01  0.00  0.00  0.00  0.00         -0.02   
                Zero          -0.04  0.01  0.00  0.01  0.01         -0.16   
Wafer           Gauss         -0.15 -0.21  0.00  0.00 -0.37          0.00   
                Inv            0.01  0.00 -0.00  0.00  0.00         -0.03   
                Opp           -0.04  0.00 -0.00  0.00  0.00         -0.02   
                SubMean        0.03  0.02 -0.01  0.04  0.04          0.01   
                Unif          -0.02  0.00  0.00  0.00  0.00          0.00   
                Zero          -0.05  0.01 -0.00  0.01  0.01          0.01   
ElectricDevices Gauss         -0.04 -0.04 -0.02 -0.03 -0.04          0.01   
                Inv           -0.02  0.01  0.05  0.01  0.03         -0.08   
                Opp           -0.03  0.10  0.00  0.10  0.16          0.04   
                SubMean        0.08  0.13  0.14  0.14  0.22          0.07   
                Unif           0.03  0.03 -0.00  0.02  0.08         -0.00   
                Zero          -0.01  0.09  0.03  0.06  0.23          0.05   

Model                                                 
Attribution                     IG    SG    GS    FO  
Dataset         Perturbation                          
FordA           Gauss         0.00 -0.00 -0.05  0.00  
                Inv           0.00 -0.00  0.00  0.00  
                Opp           0.00 -0.00  0.00  0.00  
                SubMean       0.00  0.00  0.00 -0.00  
                Unif         -0.00 -0.00  0.00 -0.00  
                Zero          0.00  0.00 -0.00 -0.00  
FordB           Gauss         0.01  0.01 -0.05  0.03  
                Inv           0.00  0.00  0.00  0.00  
                Opp           0.00 -0.04  0.00  0.01  
                SubMean       0.01 -0.13  0.01  0.01  
                Unif          0.00  0.00  0.00  0.00  
                Zero          0.01 -0.13  0.01  0.02  
Wafer           Gauss        -0.20 -0.00  0.03 -0.42  
                Inv           0.00 -0.01  0.01  0.01  
                Opp           0.00 -0.01  0.01  0.01  
                SubMean       0.05 -0.00  0.02  0.03  
                Unif          0.00 -0.01  0.01  0.01  
                Zero          0.01 -0.02  0.04  0.02  
ElectricDevices Gauss        -0.01  0.04  0.03 -0.09  
                Inv           0.02 -0.05  0.03 -0.04  
                Opp           0.14  0.04  0.12  0.12  
                SubMean       0.15  0.08  0.13  0.23  
                Unif          0.06  0.03  0.04  0.02  
                Zero          0.12  0.04  0.10  0.16

### Plots

#### Perturbation Metric Distributions

The code below reproduces Figures 3a and 3b from the paper. You can modify the parameters at the top of the code block to visualize results for different experimental conditions.

In [20]:
dataset: str = "FordB"
model: str = "InceptionTime"
perturbation_strategy: str = "SubMean"
attribution_methods = ["GR", "IG", "SG", "GS", "FO"]

# filter data for plotting
plot_data = results_df_named.copy().query(
    "perturbation_strategy == @perturbation_strategy"
    "& attribution_method in @attribution_methods"
    "& dataset_name == @dataset"
    "& model_name == @model"
)

# plot distribution of metric across attribution methods for a dataset,
# model and perturbation strategy
plot_kwargs = {
    "data": plot_data,
    "x_col": "attribution_method",
    "y_col": "ds",
    "x_levels": attribution_methods,
    "x_label": "Attribution Method",
    "y_label": "Degradation Score (DS)",
    "plot_size": (width, height),
}
ds_dist_plot = plot_metric_distributions(**plot_kwargs)
ds_dist_plot_stratified = plot_metric_distributions(
    **plot_kwargs, color_by="true_label"
)

# show plots
ds_dist_plot.show()
ds_dist_plot_stratified.show()

<path d="M310.1306996449988 313.653757452649 L310.1306996449988 313.653757452649 L310.1277740981659 313.33685452170135 L310.1251240903403 313.01995159075364 L310.12272059003465 312.703048659806 L310.1205289338559 312.3861457288583 L310.11850940838264 312.06924279791065 L310.1166179425094 311.752339866963 L310.1148068928346 311.4354369360153 L310.11302590178474 311.11853400506766 L310.1112228058888 310.80163107412 L310.10934457004885 310.48472814317233 L310.10733822283794 310.16782521222467 L310.1051517678515 309.85092228127695 L310.1027350469403 309.5340193503293 L310.10004053274224 309.2171164193816 L310.09702403025994 308.90021348843396 L310.0936452702257 308.5833105574863 L310.08986838054625 308.2664076265386 L310.08566222612325 307.9495046955909 L310.08100061164225 307.63260176464325 L310.07586234639086 307.3156988336956 L310.070231174632 306.99879590274793 L310.06409557938593 306.68189297180027 L310.0574484715011 306.3649900408526 L310.0502867795061 306.04808710990494 L310.0426109587861 305.7311841789572 L310.03442444104104 305.41428124800956 L310.02573304666555 305.0973783170619 L310.0165443835965 304.78047538611423 L310.0068672562745 304.46357245516657 L309.99671110766326 304.14666952421885 L309.98608551578894 303.8297665932712 L309.9749997640565 303.5128636623235 L309.9634625017436 303.19596073137586 L309.95148150766244 302.8790578004282 L309.93906356612547 302.56215486948054 L309.926214460182 302.2452519385329 L309.91293908273616 301.9283490075852 L309.8992416617627 301.6114460766375 L309.88512609153213 301.29454314568983 L309.8705963576911 300.97764021474217 L309.85565704033667 300.6607372837945 L309.84031387599714 300.34383435284684 L309.8245743567935 300.0269314218991 L309.8084483430853 299.71002849095146 L309.79194866468026 299.3931255600038 L309.77509168524466 299.07622262905613 L309.75789780492335 298.75931969810847 L309.74039187736264 298.4424167671608 L309.72260351929924 298.12551383621314 L309.70456729359455 297.8086109052654 L309.6863227499788 297.49170797431776 L309.6679143117407 297.1748050433701 L309.64939100104243 296.85790211242244 L309.63080600032936 296.5409991814748 L309.6122160523081 296.22409625052705 L309.5936807060289 295.9071933195794 L309.57526142158144 295.59029038863173 L309.5570205506396 295.27338745768407 L309.53902021442394 294.9564845267364 L309.52132110444677 294.63958159578874 L309.5039812345354 294.3226786648411 L309.487054674985 294.0057757338934 L309.4705903011883 293.68887280294575 L309.4546305896515 293.37196987199803 L309.4392104939151 293.05506694105037 L309.4243564315297 292.7381640101027 L309.41008541093674 292.42126107915504 L309.39640432390627 292.1043581482073 L309.3833094251951 291.78745521725966 L309.37078601640064 291.470552286312 L309.35880834575016 291.15364935536434 L309.34733972992603 290.8367464244167 L309.3363328981593 290.519843493469 L309.3257305529029 290.20294056252135 L309.3154661356024 289.8860376315737 L309.30546478059216 289.569134700626 L309.2956444351304 289.2522317696783 L309.2859171191939 288.93532883873064 L309.27619029502387 288.618425907783 L309.26636831364624 288.3015229768353 L309.2563539037773 287.98462004588765 L309.2460496677042 287.66771711493993 L309.23535954893407 287.35081418399227 L309.224190237613 287.0339112530446 L309.2124524818895 286.71700832209694 L309.2000622764604 286.4001053911493 L309.1869419034041 286.0832024602016 L309.1730208049391 285.76629952925396 L309.15823627282543 285.4493965983063 L309.1425339445889 285.1324936673586 L309.1258681024355 284.8155907364109 L309.1082017764712 284.49868780546325 L309.0895066594834 284.1817848745156 L309.0697628459135 283.8648819435679 L309.0489584126025 283.5479790126202 L309.02708886328287 283.23107608167254 L309.0041564624946 282.9141731507249 L308.9801694875114 282.5972702197772 L308.95514142889147 282.28036728882955 L308.92909017134934 281.9634643578819 L308.90203718673456 281.6465614269342 L308.8740067699954 281.3296584959865 L308.84502534709713 281.01275556503884 L308.81512088099964 280.

<path d="M281.70009216782097 269.6250062154082 L281.70009216782097 269.6250062154082 L281.7223758298504 269.42396787888435 L281.77887966700246 269.22292954236053 L281.8461562440535 269.0218912058367 L281.9030271476406 268.82085286931294 L281.9403422661163 268.6198145327891 L281.9600132214847 268.4187761962653 L281.9684872974341 268.21773785974153 L281.97149936950586 268.0166995232177 L281.9723880339946 267.8156611866939 L281.97260652110754 267.61462285017006 L281.97265140808787 267.41358451364624 L281.97265912918215 267.2125461771225 L281.9726602428002 267.01150784059865 L281.9726603776262 266.81046950407483 L281.9726603913402 266.609431167551 L281.9726603925129 266.4083928310272 L281.9726603925973 266.2073544945034 L281.9726603926024 266.0063161579796 L281.9726603926026 265.8052778214558 L281.9726603926026 265.60423948493195 L281.9726603926026 265.4032011484082 L281.9726603926026 265.20216281188436 L281.9726603926026 265.00112447536054 L281.9726603926026 264.8000861388367 L281.9726603926026 264.59904780231295 L281.9726603926026 264.39800946578913 L281.9726603926026 264.1969711292653 L281.9726603926026 263.9959327927415 L281.9726603926026 263.79489445621766 L281.9726603926026 263.5938561196939 L281.9726603926026 263.3928177831701 L281.9726603926026 263.19177944664625 L281.9726603926026 262.9907411101225 L281.9726603926026 262.78970277359866 L281.9726603926026 262.58866443707484 L281.9726603926026 262.387626100551 L281.9726603926026 262.1865877640272 L281.9726603926026 261.98554942750343 L281.9726603926026 261.7845110909796 L281.9726603926026 261.5834727544558 L281.9726603926026 261.38243441793196 L281.9726603926026 261.18139608140814 L281.9726603926026 260.9803577448844 L281.9726603926026 260.77931940836055 L281.9726603926026 260.5782810718367 L281.9726603926026 260.37724273531296 L281.9726603926026 260.1762043987891 L281.9726603926026 259.9751660622653 L281.9726603926026 259.7741277257415 L281.9726603926026 259.57308938921767 L281.9726603926026 259.3720510526939 L281.9726603926026 259.1710127161701 L281.9726603926026 258.96997437964626 L281.9726603926026 258.76893604312244 L281.9726603926026 258.5678977065986 L281.9726603926026 258.36685937007485 L281.9726603926026 258.165821033551 L281.9726603926026 257.9647826970272 L281.9726603926026 257.76374436050344 L281.9726603926026 257.5627060239796 L281.9726603926026 257.3616676874558 L281.9726603926026 257.16062935093197 L281.9726603926026 256.95959101440815 L281.9726603926026 256.7585526778844 L281.9726603926026 256.55751434136056 L281.9726603926026 256.35647600483674 L281.9726603926026 256.15543766831297 L281.9726603926026 255.95439933178912 L281.9726603926026 255.75336099526533 L281.9726603926026 255.5523226587415 L281.9726603926026 255.35128432221768 L281.9726603926026 255.15024598569389 L281.9726603926026 254.9492076491701 L281.9726603926026 254.74816931264627 L281.9726603926026 254.54713097612245 L281.9726603926026 254.34609263959865 L281.9726603926026 254.14505430307486 L281.9726603926026 253.94401596655104 L281.9726603926026 253.7429776300272 L281.9726603926026 253.5419392935034 L281.9726603926026 253.34090095697962 L281.9726603926026 253.1398626204558 L281.9726603926026 252.93882428393198 L281.9726603926026 252.73778594740818 L281.9726603926026 252.53674761088436 L281.9726603926026 252.33570927436054 L281.9726603926026 252.13467093783674 L281.9726603926026 251.93363260131292 L281.9726603926026 251.73259426478913 L281.9726603926026 251.53155592826533 L281.9726603926026 251.3305175917415 L281.9726603926026 251.1294792552177 L281.9726603926026 250.92844091869387 L281.9726603926026 250.72740258217007 L281.9726603926026 250.52636424564628 L281.9726603926026 250.32532590912245 L281.9726603926026 250.12428757259863 L281.9726603926026 249.92324923607484 L281.9726603926026 249.72221089955104 L281.9726603926026 249.52117256302722 L281.9726603926026 249.3201342265034 L281.9726603926026 249.1190958899796 L281.9726603926026 248.9180575534558 L281.9726603926026 248.717019216932 L281.972

#### Parallel Coordinates Plot

The following code reproduces Figures 4a and 4b. You can adjust the parameters to visualize different experimental configurations.

In [21]:
dataset: str = "FordB"
model: str = "InceptionTime"
attribution_method: str = "FO"
color_by: str = "true_label"


def filter_and_format_plot_data(df) -> pd.DataFrame:
    """Convenience function to filter and format data for plotting"""
    filter_str = (
        "attribution_method == @attribution_method"
        "& dataset_name == @dataset"
        "& model_name == @model"
    )
    plot_data = df.copy().query(filter_str)
    plot_data.loc[:, label] = plot_data[color_by].astype(str)
    plot_data = plot_data.sort_values(color_by)
    return plot_data


plot_data_named = filter_and_format_plot_data(results_df_named)
plot_data_constant = filter_and_format_plot_data(results_df_constant)

# plot parallel coordinates for perturbation strategies
plot_kwargs_parallel = {
    "x_col": "perturbation_strategy",
    "x_label": "Perturbation Strategy",
    "y_col": "ds",
    "y_label": "Degradation Score (DS)",
    "group_col": color_by,
    "instance_id_col": "idx",
    "plot_size": (width, height),
}
pc_plot_named = plot_parallel_coords(
    data=plot_data_named, order="ascending", **plot_kwargs_parallel
)
pc_plot_named.show()

pc_plot_constant = plot_parallel_coords(
    data=plot_data_constant, order="name", **plot_kwargs_parallel
)
pc_plot_named.show()

<path d="M22.98660135671619 297.81445148683144 L22.98660135671619 297.81445148683144 L137.91960814029713 295.43980727656606 L252.85261492387806 285.04273725070493 L367.785621707459 244.86236588337684 L482.7186284910399 253.33635

<path d="M22.98660135671619 297.81445148683144 L22.98660135671619 297.81445148683144 L137.91960814029713 295.43980727656606 L252.85261492387806 285.04273725070493 L367.785621707459 244.86236588337684 L482.7186284910399 253.33635

#### Example of Desirable MoRF and LeRF Perturbation Curves

This following code reproduces Figure 2, showing the ideal pattern where:
- MoRF curve (perturbing important features) drops rapidly
- LeRF curve (perturbing unimportant features) remains stable initially
- The gap between curves (DS) is substantial


In [22]:
def create_curve(
    n_points: int,
    curve_type: str = "quick_drop",
    params: dict = None,
    noise_level: float = 0.005,
) -> np.ndarray:
    """Create a perturbation curve with specified characteristics.

    Args:
        n_points: Number of points to generate
        curve_type: Either "quick_drop" or "delayed_drop"
        params: Parameters specific to the curve type
        noise_level: Amount of noise to add

    Returns:
        np.ndarray: Perturbation curve values
    """
    # Default parameters
    params = params or {}
    curve = np.full(n_points, 0.05)

    if curve_type == "quick_drop":
        # Quick drop curve (MoRF)
        drop_idx = int(params.get("drop_end", 0.25) * n_points)
        if drop_idx > 0:
            x_drop = np.linspace(0, np.pi, drop_idx)
            curve[:drop_idx] = 0.95 - 0.9 * (1 - np.cos(x_drop)) / 2

    elif curve_type == "delayed_drop":
        # Delayed drop curve (LeRF)
        start_idx = int(params.get("start_drop", 0.5) * n_points)
        end_idx = int(params.get("end_drop", 0.8) * n_points)
        curve[:start_idx] = 0.95

        # Create sigmoid drop between start and end
        if start_idx < end_idx:
            x_drop = np.linspace(0, 1, end_idx - start_idx)
            steepness = params.get("steepness", 10)
            curve[start_idx:end_idx] = 0.95 - 0.9 / (
                1 + np.exp(-steepness * (x_drop - 0.5))
            )

    # Add noise if specified
    if noise_level > 0:
        curve += noise_level * np.random.randn(n_points)

    # Ensure exact endpoints and valid range
    curve[0] = 0.95
    curve[-1] = 0.05
    return np.clip(curve, 0.05, 0.95)


# Generate example curves
np.random.seed(42)
n_points = 100
morf_curve = create_curve(n_points, "quick_drop", {"drop_end": 0.3}, noise_level=0)
lerf_curve = create_curve(
    n_points, "delayed_drop", {"start_drop": 0.4, "end_drop": 0.85}, noise_level=0
)

# Create result object and plot
high_ds_example = PerturbationResult(
    scores={"MoRF": morf_curve.tolist(), "LeRF": lerf_curve.tolist()},
    auc_scores={"MoRF": 0.3, "LeRF": 0.7},
)

# Plot perturbation curves
ds_method_example_plot = plot_perturbation_analysis_curves(
    results=high_ds_example,
    title="",
    plot_between=True,
    perturbation_ratio=1,
    plot_size=(width, height),
)
ds_method_example_plot += theme(
    legend_position=[0.505, 1],
    legend_justification=[0, 1],
    legend_direction="horizontal",
    legend_background=element_rect(color="grey", size=0.1),
) + labs(color="Perturbation Order", fill="Perturbation Type", title="")
ds_method_example_plot.show()

<path d="M28.573583731218424 30.92233220847396 L28.573583731218424 30.92233220847396 L34.34602488904033 30.92233220847396 L40.11846604686223 30.92233220847396 L45.89090720468414 30.92233220847396 L51.66334836250604 30.92233220847396 L57.435789520327944 30.92233220847396 L63.208230678149846 30.92233220847396 L68.98067183597175 30.92233220847396 L74.75311299379366 30.92233220847396 L80.52555415161557 30.92233220847396 L86.29799530943747 30.92233220847396 L92.07043646725937 30.92233220847396 L97.84287762508127 30.92233220847396 L103.61531878290317 30.92233220847396 L109.38775994072508 30.92233220847396 L115.16020109854698 30.92233220847396 L120.9326422563689 30.92233220847396 L126.7050834141908 30.92233220847396 L132.4775245720127 30.92233220847396 L138.2499657298346 30.92233220847396 L144.02240688765653 30.92233220847396 L149.79484804547843 30.92233220847396 L155.56728920330033 30.92233220847396 L161.33973036112224 30.92233220847396 L167.11217151894414 30.92233220847396 L172.88461267676604 30.92233220847396 L178.65705383458794 30.92233220847396 L184.42949499240984 30.92233220847396 L190.20193615023175 30.92233220847396 L195.97437730805365 30.92233220847396 L201.74681846587555 30.92233220847396 L207.51925962369745 30.92233220847396 L213.29170078151938 30.92233220847396 L219.06414193934128 30.92233220847396 L224.83658309716319 30.92233220847396 L230.6090242549851 30.92233220847396 L236.381465412807 30.92233220847396 L242.1539065706289 30.92233220847396 L247.9263477284508 30.92233220847396 L253.6987888862727 30.92233220847396 L259.4712300440946 32.835171271193076 L265.2436712019165 33.31918110181283 L271.01611235973843 33.92436591892704 L276.78855351756033 34.68032877304847 L282.56099467538223 35.62349132979284 L288.33343583320413 36.79843236043928 L294.10587699102604 38.25935700215905 L299.87831814884794 40.071627280741154 L305.65075930666984 42.3132235757156 L311.42320046449174 45.075918830916976 L317.19564162231364 48.46582853922928 L322.96808278013555 52.60285245507117 L328.74052393795745 57.618364941794994 L334.51296509577935 63.65038015230374 L340.28540625360125 70.83539088191799 L346.05784741142315 79.29626949213159 L351.83028856924506 89.12616079065958 L357.60272972706696 100.36929033126395 L363.37517088488886 113.0010183530265 L369.14761204271076 126.9109870509329 L374.92005320053266 141.8942163215687 L380.69249435835457 157.65470976286704 L386.46493551617647 173.82399581631248 L392.23737667399837 189.99328186975785 L398.0098178318203 205.75377531105622 L403.78225898964223 220.73700458169208 L409.55470014746413 234.64697327959846 L415.32714130528603 247.27870130136097 L421.09958246310794 258.5218308419653 L426.87202362092984 268.3517221404933 L432.64446477875174 276.81260075070696 L438.41690593657364 283.9976114803212 L444.18934709439554 290.0296266908299 L449.96178825221745 295.0451391775538 L455.73422941003935 299.1821630933957 L461.50667056786125 302.5720728017079 L467.27911172568315 305.3347680569093 L473.05155288350505 307.5763643518838 L478.82399404132696 309.3886346304659 L484.59643519914886 310.8495592721856 L490.3688763569708 312.02450030283205 L496.1413175147927 312.9676628595764 L501.9137586726146 313.72362571369786 L507.6861998304365 314.3288105308122 L513.4586409882584 314.8128203614319 L519.2310821460803 316.725659424151 L525.0035233039022 316.725659424151 L530.7759644617241 316.725659424151 L536.548405619546 316.725659424151 L542.3208467773679 316.725659424151 L548.0932879351898 316.725659424151 L553.8657290930117 316.725659424151 L559.6381702508336 316.725659424151 L565.4106114086555 316.725659424151 L571.1830525664774 316.725659424151 L576.9554937242993 316.725659424151 L582.7279348821212 316.725659424151 L588.500376039943 316.725659424151 L594.272817197765 316.725659424151 L600.0452583555868 316.725659424151 L600.0452583555868 316.725659424151 L594.272817197765 316.725659424151 L588.500376039943 316.725659424151 L582.7279348821212 316.725659424151 L576.9554937242993 316.725659424151 L571.1830525664774 316